# Make blast
***

In [ ]:
# *******************************************************************************************************************************************
# 2. The Blastp analysis : preparation step 
# *******************************************************************************************************************************************

    # The makeblastdb command : 
makeblastdb -in /home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/bea_all_depolymerase.db/bead_depo/bea_depo.DB.fasta \
-dbtype prot  \
-title depolymerase_db \
-out /home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/bea_all_depolymerase.db/bead_depo/bea_depo.DB


    # The command for the multifasta faa for the congruent and the loners :
import os
import subprocess
from multiprocessing.pool import ThreadPool

path_70="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
path_db = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/bea_all_depolymerase.db/bead_depo/bea_depo.DB"
path_out = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/blast_out_phageboost_70_80"

path_fasta = [f"{path_70}/{strain}/tmp/{phage}/{fasta}" 
              for strain in os.listdir(f"{path_70}") 
              for phage in os.listdir(f"{path_70}/{strain}/tmp") 
              for fasta in os.listdir(f"{path_70}/{strain}/tmp/{phage}")]

def blast_bea(path) :
    #path_out = "/".join(path.split("/")[0:-1])
    file_name = path.split("/")[-1].split(".fasta")[0]
    blastp = f"blastp -query {path}  -db {path_db} -out {path_out}/{file_name}.out -outfmt 6"
    sub_blastp = subprocess.Popen(blastp, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    press_out, press_err = sub_blastp.communicate()
    print(press_out)
    
if __name__ == '__main__':
    with ThreadPool(10) as p:
        p.map(blast_bea, path_fasta)
        
        
def show_result(protein) :
    outfmt = ["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart" ,"send", "evalue", "bitscore"]
    strain = protein.split("__")[0]
    prophage = protein.split("__")[1]
    results = pd.read_csv(f"{path_out}/{protein}.out", names = outfmt, sep = "\t")
    return results
        
        

# ****************************************************************************************************
#!/bin/bash
#BATCH --job-name=blast_depo
#SBATCH --qos=medium 
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=40
#SBATCH --mem=50gb 
#SBATCH --time=2-00:00:00 
#SBATCH --output=blast_depo%j.log 

module restore la_base
conda activate blast_life

python /home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/bea_all_depolymerase.db/script_files/blast.full_phageboost.py 


In [ ]:
import os
import subprocess
from multiprocessing.pool import ThreadPool

path_70="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
path_db = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/bea_all_depolymerase.db/bead_depo/bea_depo.DB"
path_out = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/blast_out_phageboost_70_80"

n = 0




***
### Scan the results :




In [ ]:
import os
import subprocess
from multiprocessing.pool import ThreadPool
import pandas as pd
from tqdm import tqdm

path_70="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
path_db = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/bea_all_depolymerase.db/bead_depo/bea_depo.DB"
path_out = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/blast_out_phageboost_70_80"
path_write = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/bea_all_depolymerase.db"
path_work = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher"


outfmt = ["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart" ,"send", "evalue", "bitscore"]
depo_prot = set()
def scan_blast(path_results) :
    with open(f"{path_work}/full_prediction.vsBeaDB.txt", "a+") as outfile :
        if os.path.getsize(path_results) >0 :
            results = pd.read_csv(f"{path_results}", names = outfmt, sep = "\t")
            if results.iloc[0]["bitscore"] > 20 :
                depo_prot.add(results.iloc[0]["qseqid"])
                outfile.write(f"{results.iloc[0]['qseqid']}\t{results.iloc[0]['bitscore']}\n")
            
            
paths = [f"{path_out}/{file}" for file in os.listdir(path_out)]        
        
if __name__ == '__main__':
    with ThreadPool(30) as p:
        p.map(scan_blast, paths)
        
    
# ****************************************************************************************************
#!/bin/bash
#BATCH --job-name=blast_depo_scan__
#SBATCH --qos=medium 
#SBATCH --ntasks=1 
#SBATCH --cpus-per-task=30
#SBATCH --mem=10gb 
#SBATCH --time=2-00:00:00 
#SBATCH --output=blast_depo_scan__%j.log 

module restore la_base
conda activate blast_life

python /home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/script_file/py_files/blastp_scan.py
 

In [ ]:
import os
import subprocess
from multiprocessing.pool import ThreadPool
import pandas as pd
from tqdm import tqdm

path_70="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
path_out = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/blast_out_phageboost_70_80"
path_write = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/bea_all_depolymerase.db"
path_work = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher"

path_db = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session"
path_phageboost = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
path_store = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session/Phrogs_prediction_20042023"

path_fastani="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_similarity/phageboost/fastANI_20102022_out"
path_phages = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_similarity/phageboost/fastANI_20102022"
path_ktype="/home/conchae/prediction_depolymerase_tropism"


cluster_df = pd.read_csv(f"{path_fastani}/clusters_99_80.clean.2004.tsv", header = 0, sep="\t")

desirable_df = pd.read_csv(f"{path_db}/Results_III_DataFrame.v3.final.desired_ipr.csv", sep="\t", header= 0)
desirable_df = desirable_df.drop_duplicates(subset = ["protein_name"], keep = "first")

results_df = pd.read_csv(f"{path_db}/Results_III_DataFrame.v3.final.csv", sep="\t", header= 0)
results_df = results_df.drop_duplicates(subset = ["protein_name"], keep = "first")


hits_blast = pd.read_csv(f"{path_work}/full_prediction.vsBeaDB.txt", sep = "\t", names = ["protein", "bitscore"])
hits_blast_75 = hits_blast[hits_blast["bitscore"] >=75]
hits_blast_100 = hits_blast[hits_blast["bitscore"] >=100]

to_decipher = set()
inside = set()
for _,row in tqdm(hits_blast_75.iterrows()) :
    if row["protein"] not in results_df["protein_name"].values :
        to_decipher.add(row["protein"])
    else :
        inside.add(row["protein"])

# Check how many passed that wouldnt have :       
got_range = set()
for _,row in tqdm(desirable_df.iterrows()) :
    if row["protein_name"] not in hits_blast_75["protein"].values :
        got_range.add(row["protein_name"])

dico_done = {}
for protein in tqdm(got_range) :
    dico_done[protein] = get_seq(protein)

ranged_sequences = set()
for protein in tqdm(dico_done) :
    ranged_sequences.add(dico_done[protein])

> Little parameters and generate the new mini batch 

In [ ]:
def get_seq(protein) :
    strain = protein.split("__")[0]
    prophage = protein.split("__")[1]
    seq = open(f"{path_70}/{strain}/tmp/{prophage}/{protein}.fasta").read().split("\n")[1]
    return seq 

In [ ]:
dico_sequences = {}
for protein in tqdm(to_decipher) :
    dico_sequences[protein] = get_seq(protein)
    
seq_to_prot = {}
for protein in dico_sequences :
    if dico_sequences[protein] not in seq_to_prot :
        tmp = []
        tmp.append(protein)
        seq_to_prot[dico_sequences[protein]] = tmp
    else :
        seq_to_prot[dico_sequences[protein]].append(protein)
                
with open(f"{path_db}/new_mini_batch.prophage_raw.json", "w") as outfile:
    json.dump(seq_to_prot, outfile)

mini_batch_dico = json.load(open(f"{path_db}/new_mini_batch.prophage_raw.json"))

mini_batch_clean = {}
for seq in mini_batch_dico :
    if len(seq) <= 1500 :
        mini_batch_clean[seq] = mini_batch_dico[seq]
        
# Write the new_mini batch : 
with open(f"{path_db}/minibatch_index.csv", "w") as outfile :
    for index,seq in enumerate(list(mini_batch_clean.keys())) :
        outfile.write(f"{index}\t{seq}\n")
        
# ==> Make prediction !
rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/15122022_session/minibatch_index.csv \
/media/concha-eloko/Linux/PPT_clean/

> Interesting lengths

In [ ]:
# Interesting lengths : 
    
lengths = []
for seq in seq_to_prot :
    lengths.append(len(seq))

b_1500 = []
for seq in seq_to_prot :
    if len(seq) < 1500 :
        b_1500.append(len(seq))

In [ ]:
# Get dico with prophage ID : index new batch : 
obs_dico = {}
for seq,list_pro in tqdm(seq_to_prot.items()):
    prophages_obs = set()
    for protein in list_pro :
        prophage = f"{'__'.join(protein.split('__')[0:-1])}.fasta"
        print(prophage)
        for index, row in cluster_df.iterrows() :
            if prophage in row["Members"].split(",") :
                prophages_obs.add(f"prophage_{index}")
                break
        continue
    obs_dico[seq] = prophages_obs             
            
for key in obs_dico:
    obs_dico[key] = list(obs_dico[key])
       
with open(f"{path_db}/new_mini_batch.prophage_indices.json", "w") as outfile:
    json.dump(obs_dico, outfile)
    
mini_batch_dico = json.load(open(f"{path_db}/new_mini_batch.prophage_indices.json"))

In [ ]:
# Write the new_mini batch : 
with open(f"{path_db}/minibatch_index.fasta", "w") as outfile :
    for seq in mini_batch_clean

In [ ]:
final_df = pd.read_csv(f"{path_db}/prophages.strain_depo.matrix.csv", header = 0 , sep="\t")

In [ ]:
import pprint
pp = pprint.PrettyPrinter(width = 50)

pp.pprint(seq_to_prot)  

In [ ]:
to_decipher = set()
inside = set()
for _,row in tqdm(desirable_df.iterrows()) :
    if row["protein_name"] not in hits_blast["protein"].values :
        to_decipher.add(row["protein_name"])
    else :
        inside.add(row["protein_name"])


In [ ]:
# How many different sequences ? :


def get_seq(protein) :
    strain = protein.split("__")[0]
    prophage = protein.split("__")[1]
    seq = open(f"{path_70}/{strain}/tmp/{prophage}/{protein}.fasta").read().split("\n")[1]
    return seq 

sequences = set()
for protein in to_decipher : 
    sequences.add(get_seq(protein))
    
    GCF_900176555.1__phage4__1078

to_predict = set()
to_check = set()
for protein in to_decipher :
    strain = protein.split("__")[0]
    prophage = protein.split("__")[1]
    if os.path.isfile(f"{path_70}/{strain}/DBsuite_depo_v3/{prophage}/{protein}.suite.hhr") == False :
        to_predict.add(protein)
    else :
        to_check.add(protein)
    